In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,KFold,LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Mengimport data

In [17]:
dataset = pd.read_csv('titanic.csv')
dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [18]:
# Convert categorical variables to numeric
le = LabelEncoder()
dataset['Sex'] = le.fit_transform(dataset['Sex'])
dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,S


In [19]:
train_data = dataset[['Sex','Age','Pclass','Fare']]

mean_age_by_class = dataset.groupby('Survived')['Age'].transform('mean')
mean_fare_by_class = dataset.groupby('Survived')['Fare'].transform('mean')
train_data['Age'] = train_data['Age'].fillna(mean_age_by_class)
train_data['Fare'] = train_data['Fare'].fillna(mean_fare_by_class)
train_data = train_data.dropna()
train_data

C:\Users\BAGAS\AppData\Local\Temp\ipykernel_7940\2524018443.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Age'] = train_data['Age'].fillna(mean_age_by_class)
C:\Users\BAGAS\AppData\Local\Temp\ipykernel_7940\2524018443.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Fare'] = train_data['Fare'].fillna(mean_fare_by_class)


,Sex,Age,Pclass,Fare
0,1,34.500000,3,7.8292
1,0,47.000000,3,7.0000
2,1,62.000000,2,9.6875
3,1,27.000000,3,8.6625
4,0,22.000000,3,12.2875
...,...,...,...,...
413,1,30.272732,3,8.0500
414,0,39.000000,1,108.9000
415,1,38.500000,3,7.2500
416,1,30.272732,3,8.0500


In [20]:
label = dataset.loc[:,['Survived']]
label = label.dropna()
label.shape

(418, 1)

# MinMax Scaler

In [21]:
scaler = MinMaxScaler()
train_data = pd.DataFrame(scaler.fit_transform(train_data), columns=train_data.columns)
print('Train Data After Norm:\n',train_data)
print("Nilai Minimum dari Age: {}".format(train_data['Age'].min()))
print("Nilai Maximum dari Age: {}".format(train_data['Age'].max()))
print("Nilai Minimum dari Fare: {}".format(train_data['Fare'].min()))
print("Nilai Maximum dari Fare: {}".format(train_data['Fare'].max()))

Train Data After Norm:
      Sex       Age  Pclass      Fare
0    1.0  0.452723     1.0  0.015282
1    0.0  0.617566     1.0  0.013663
2    1.0  0.815377     0.5  0.018909
3    1.0  0.353818     1.0  0.016908
4    0.0  0.287881     1.0  0.023984
..   ...       ...     ...       ...
413  1.0  0.396977     1.0  0.015713
414  0.0  0.512066     0.0  0.212559
415  1.0  0.505473     1.0  0.014151
416  1.0  0.396977     1.0  0.015713
417  1.0  0.396977     1.0  0.043640

[418 rows x 4 columns]
Nilai Minimum dari Age: 0.0
Nilai Maximum dari Age: 1.0
Nilai Minimum dari Fare: 0.0
Nilai Maximum dari Fare: 1.0


In [22]:
X = train_data
Y = label
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

HOLD OUT CROSS VALIDATION atau TRAIN-TEST SPLIT

In [23]:
y_train = np.ravel(y_train)
logreg.fit(x_train, y_train)
predict = logreg.predict(x_test)

print("Accuracy training set: {}".format(accuracy_score(logreg.predict(x_train),y_train)))
print("Accuracy test set: {}".format(accuracy_score(predict,y_test)))

Accuracy training set: 1.0
Accuracy test set: 1.0


K_FOLD CROSS VALIDATION

In [24]:
kf = KFold(n_splits=10)
score = cross_val_score(logreg,X,Y,cv=kf)

c:\Users\BAGAS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\BAGAS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\BAGAS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\BAGAS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataC

In [25]:
print("Cross Valdation Score: {}".format(score))
print("Average Cross Validation Score: {}".format(score.mean()))

Cross Valdation Score: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Average Cross Validation Score: 1.0


LEAVE ONE OUT CROSS VALIDATION

In [26]:
loo = LeaveOneOut()

loo.get_n_splits(X)

tree = RandomForestClassifier(n_estimators=10, max_depth=5, n_jobs=1)

score= cross_val_score(tree,X,Y,cv=loo)

c:\Users\BAGAS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


c:\Users\BAGAS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\BAGAS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\BAGAS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\BAGAS\AppData\Local\Programs\Py

In [27]:
print("Cross Valdation Score: {}".format(score))
print("Average Cross Validation Score: {}".format(score.mean()))

Cross Valdation Score: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 

# K-NN Classification

In [28]:
kNN = KNeighborsClassifier(n_neighbors=3)
kNN.fit(train_data.dropna(),np.ravel(Y))
class_result = kNN.predict(x_test)
error = 1 - accuracy_score(y_test, class_result)
print(f'error k=3: {error}')

error k=3: 0.0
